<a href="https://colab.research.google.com/github/EtzionR/LM4GeoAI/blob/main/Geo_LLM_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ex 3: Basic Geo Agents and RAG

### created by Etzion Harari | Geo-AI Course

[**https://github.com/EtzionR/LM4GeoAI**](https://github.com/EtzionR/LM4GeoAI)

## Prerequisites: pip install wikipedia, cohere & osmnx

In [43]:
print('Install wikipedia, cohere & osmnx using pip...\n')

!pip install -q wikipedia cohere osmnx

print('\nDone!')

Install wikipedia, cohere & osmnx using pip...


Done!


## Imports

In [44]:
from difflib import get_close_matches
from getpass import getpass

import numpy as np
import osmnx as ox
import wikipedia
import folium
import cohere

## Define cohere model

You can find documentation of cohere models here:

 [https://docs.cohere.com/docs/models#command](https://docs.cohere.com/docs/models#command)



In [45]:
MODEL  = "command-a-03-2025"
TEMP = .01

## Enter cohere API key

You can get your cohere API Key here (you should log-in / sign-in to cohere):

[https://dashboard.cohere.com/api-keys](https://dashboard.cohere.com/api-keys)

In [46]:
KEY = getpass("Please Enter COHERE KEY:\n")

len(KEY)

Please Enter COHERE KEY:
··········


40

## Init cohere Client

In [47]:
client = cohere.Client(api_key=KEY)
client

## Example of calling to cohere model

In [48]:
QUERY = 'What is panda bear?' # an example of a simple query

output = client.chat(model=MODEL,
                     message=QUERY,
                     temperature=TEMP).text

print(output)

A panda bear, commonly referred to as a giant panda, is a distinctive black-and-white bear native to central China. Scientifically known as *Ailuropoda melanoleuca*, it is one of the most recognizable and beloved animals in the world. Here are some key facts about panda bears:

1. **Appearance**: Pandas are known for their striking black-and-white fur, with black patches around the eyes, ears, and across their bodies, contrasting with white fur elsewhere.

2. **Diet**: Despite being classified as carnivores, pandas primarily eat bamboo, consuming up to 15-30 pounds (7-14 kg) of it daily. They also occasionally eat other vegetation, small animals, and fish.

3. **Habitat**: Pandas live in temperate forests in the mountainous regions of China, primarily in Sichuan, Shaanxi, and Gansu provinces. They prefer areas with dense bamboo growth.

4. **Behavior**: Pandas are generally solitary and spend much of their time eating and resting. They are excellent climbers and can also swim.

5. **Co

## Write a function that for given location adress can load every POI place type from OSM

In [49]:
def get_pois_near_adress(location_adress, place_type, radius=500):
    """
    Load OSM POIs from given place_type

    :location_adress: location adress for searching OSM
    :place_type: type of point of interest (POI)
    :radius: radius around location_adress (in meters, int, default: 500)

    :return: selected dataframe
    """

    # Load data from OSM
    out = ox.features_from_address(location_adress, {"amenity": True}, radius)
    out = out[['name','amenity','geometry']]
    out = out.rename(columns={'amenity':'place_type'})

    # Find the closets place_type (using SequenceMatcher)
    place_type = get_close_matches(place_type,
                                   [*out.place_type.unique()],
                                   n=1,
                                   cutoff=.0)[0]

    # select the place_type
    dataframe = out[out.place_type==place_type]

    return dataframe

# ------------------------------------------------------------------------------


# get_place_info application:

location_adress = 'מגדלי עזריאלי'
place_type      = 'fual'           # not fuel!

get_pois_near_adress(location_adress, place_type)

name place_type                   geometry
element id                                                  
node    442893138  דלק       fuel  POINT (34.79586 32.07468)
        442893144  דלק       fuel  POINT (34.79449 32.07144)
        442898671   פז       fuel   POINT (34.79724 32.0719)

## Write a function that that calculate the driving path from given adress to second adress.
The path should be caluclated using the OSM roads network

In [50]:
def shortest_path_between_adresses(adress_1, adress_2,  gap=.25):
    """
    Calculate shortest path between two adresses using OSM roads network

    :adress_1: origin adress (source)
    :adress_2: destination adress (target)
    :gap: buffer ratio from bbox (float, default: .25)

    :return: shortest path between adress_1 to adress_2 (coordinates)
    """

    # gecode to both adresses
    y1, x1 = ox.geocode(adress_1)
    y2, x2 = ox.geocode(adress_2)

    # build the bbox
    delta = max(abs(x1-x2)*gap, abs(y1-y2)*gap)
    bbox = [min(x1,x2)-delta, min(y1,y2)-delta, max(x1,x2)+delta, max(y1,y2)+delta]

    # get the roads network graph
    G = ox.graph.graph_from_bbox(bbox, truncate_by_edge =True, network_type ='drive', simplify =False)

    # find the origin & destination nodes
    node1 = ox.distance.nearest_nodes(G, x1, y1)
    node2 = ox.distance.nearest_nodes(G, x2, y2)

    # calculate path between origin to destination
    route = ox.routing.shortest_path(G, orig=node1, dest=node2)
    route_xy = [(G.nodes[osmid]['x'], G.nodes[osmid]['y']) for osmid in route]

    return route_xy

# ------------------------------------------------------------------------------


# shortest_path_between_adresses application:

ad1 = "אוניברסיטת תל אביב"
ad2 = 'מגדלור רידינג'

path = shortest_path_between_adresses(ad1, ad2)

print(f'Number of nodes in the output path: {len(path)}\n\nFirst 5 points:\n{path[:5]}')

Number of nodes in the output path: 128

First 5 points:
[(34.8037004, 32.1107766), (34.8037061, 32.1107118), (34.8037047, 32.1105204), (34.8036907, 32.1103309), (34.8036844, 32.1101144)]


## Write the **system prompt** and **tools schema** for cohere client

Cohere MCP documentation:

[https://docs.cohere.com/page/basic-tool-use](https://docs.cohere.com/page/basic-tool-use)

In [51]:
# system prompt
system_prompt = """You are an AI assistant with access to the following tools:
get_pois_near_adress tool, which enable you to run osm query.
shortest_path_between_adresses tool, which enable you to calculate route between to given place names."""

# tools schema
tools = [
    {
        "name": "get_pois_near_adress",
        "description": "can return every type of geographic location near some given adress",
        "parameter_definitions": {
            "location_adress": {
                "description": "free text location adress",
                "type": "str",
                "required": True
            },
            "place_type": {
                "description": "type of point of interest (POI)",
                "type": "str",
                "required": True
            },
            "radius": {
                "description": "radius of search in meters",
                "type": "float",
                "required": False
            }
        }
    },

    {
        "name": "shortest_path_between_adresses",
        "description": "Calculate shortest path between two adresses using OSM roads network",
        "parameter_definitions": {
            "adress_1": {
                "description": "adress of the origin point (free text)",
                "type": "str",
                "required": True
            },
            "adress_2": {
                "description": "adress of the destination point (free text)",
                "type": "str",
                "required": True
            },
            "gap": {
                "description": "buffer ratio of the bbox to load roads network",
                "type": "float",
                "required": False
            }
        }
    },
]

# tools map
tools_map = {get_pois_near_adress.__name__: get_pois_near_adress,
                 shortest_path_between_adresses.__name__: shortest_path_between_adresses}
tools_map

{'get_pois_near_adress': <function __main__.get_pois_near_adress(location_adress, place_type, radius=500)>,
 'shortest_path_between_adresses': <function __main__.shortest_path_between_adresses(adress_1, adress_2, gap=0.25)>}

## Example call for calling cohere client while using **get_pois_near_adress** tool

In [52]:
user_prompt = f"I want you to return every cafe near Azrieli Center"


response = client.chat(
    model = MODEL,
    message = user_prompt,
    preamble = system_prompt,
    tools = tools
)

print(f"Model output: {response.text}\n\n")

out = tools_map[response.tool_calls[0].name](**response.tool_calls[0].parameters)
out

Model output: I will use the get_pois_near_adress tool to find every cafe near Azrieli Center.




name place_type                   geometry
element id                                                            
node    553022802       קפה שרגא       cafe    POINT (34.7903 32.0707)
        1207157419           Uno       cafe  POINT (34.78908 32.07748)
        3465156312        רולדין       cafe  POINT (34.78693 32.07173)
        6469411087        לנדוור       cafe  POINT (34.79164 32.07452)
        6469411088       קפה קפה       cafe   POINT (34.79172 32.0747)
        6644165508         Aroma       cafe  POINT (34.78769 32.07308)
        7211406506   Max Brenner       cafe   POINT (34.78677 32.0715)
        10784964980      Arcaffe       cafe  POINT (34.78864 32.07829)
        10812416453       BEITEA       cafe  POINT (34.78798 32.07223)
        11802167414      Arcaffe       cafe  POINT (34.79304 32.07731)
        12032015683          mae       cafe  POINT (34.78829 32.07195)
        12139988888     קפה עוטף       cafe  POINT (34.78739 32.07308)
        12635329119       רולדין       cafe   POINT (34.79232 32.0743)
        13338979218         Biga       cafe  POINT (34.78691 32.07308)

## Display tool outputs on folium map

In [53]:
fmap = folium.Map(location=[out.geometry.y.median(), out.geometry.x.median()],
                  zoom_start=15.5, width=500, height=500)

folium.GeoJson(out).add_to(fmap)

fmap

## Example call for calling cohere client while using **shortest_path_between_adresses** tool

In [54]:
user_prompt = f"אני רוצה לדעת מה המסלול מחוף מציצים למגדלי עזריאלי"


response = client.chat(
    model = MODEL,
    message = user_prompt,
    preamble = system_prompt,
    tools = tools
)

print(f"Model output: {response.text}\n\n")

out = tools_map[response.tool_calls[0].name](**response.tool_calls[0].parameters)

print(f'Output length: {len(out)}')

Model output: אשתמש בכלי shortest_path_between_adresses כדי למצוא את המסלול הקצר ביותר מחוף מציצים למגדלי עזריאלי.


Output length: 137


## Display tool outputs on folium map

In [55]:


fmap = folium.Map(location=[np.array(out)[:,1].mean(), np.array(out)[:,0].mean()],
                  zoom_start=14, width=500, height=500)

folium.PolyLine(
    locations=[(y,x) for x,y in out],
    color="blue",
    weight=5,
    tooltip=user_prompt).add_to(fmap)

folium.Marker([out[0][-1] ,out[0][0]],
                  tooltip='Start',
                  zIndexOffset=1,
                  icon=folium.Icon(color='blue', icon='play')).add_to(fmap)

folium.Marker([out[-1][-1] ,out[-1][0]],
                  tooltip='End',
                  zIndexOffset=1,
                  icon=folium.Icon(color='blue', icon='pause')).add_to(fmap)

fmap

## Write a simple RAG *(Retrieved Augmented Generation)* that can describe the environment of given coordinate

In [58]:

def get_pois_from_coordinates(x,y, poi_search_radius=100):
    """
    Load all OSM POIs in a given radius around given point

    :x: x coordiante (WGS84 GEO DD EPSG:4326, float)
    :y: y coordiante (WGS84 GEO DD EPSG:4326, float)
    :dist: distance around the point in meters (float | int)

    return: all place name in the given radius
    """

    point = (y,x)

    options = ox.features.features_from_point(center_point = point, dist=poi_search_radius, tags = {'building': True, "amenity": True})
    options = options[['name']].dropna().drop_duplicates()

    return [*options.name]

def get_context_from_wikipedia(page_name, context_length=256):
    """
    Load the content of wikipedia page by given page name

    :page_name: given wikipedia page name (str)
    :context_length: maximum context length (int)

    return: context knowledge on the page name
    """

    page = wikipedia.page(wikipedia.search(page_name)[0])

    return page.content[:context_length]

def describe_location(x,y, poi_search_radius=100):
    """
    describe a location given its coordinates

    :x: x coordiante (WGS84 GEO DD EPSG:4326, float)
    :y: y coordiante (WGS84 GEO DD EPSG:4326, float)
    :dist: distance around the point in meters (float | int)

    return: cohere model description on the location
    """

    place_names = get_pois_from_coordinates(x,y, poi_search_radius=poi_search_radius)
    context = get_context_from_wikipedia(place_names[0])

    response = client.chat(model = MODEL,
                           message = f'Please describe {place_names[0]} given the context knowledge from wikipedia:\n\n{context}')
    return response.text

output = describe_location(-77.0352332, 38.8893845)

print(output)

The **Washington Monument** is an iconic 555-foot (169 m) tall marble, granite, and sandstone obelisk located on the **National Mall** in **Washington, D.C.**, United States. It was constructed to honor **George Washington**, a **Founding Father** of the United States and the nation's **first president**. Standing prominently east of the **Reflecting Pool** and the **Lincoln Memorial**, the monument is a central feature of the National Mall and a symbol of American history and democracy.

Construction of the monument began in **1848**, but it was halted due to funding issues and the American Civil War, and was not completed until **1884**. It was officially opened to the public in **1888**. Designed by architect **Robert Mills**, the obelisk's simple yet striking design reflects classical Egyptian style, with a towering structure that tapers toward the top. The monument is constructed with marble, granite, and sandstone, and its exterior features two shades of stone due to the construc